In [1]:
import dimcli
import json
import math
import time
import pandas as pd

In [2]:
dimcli.login()

DimCli v0.6.1 - Succesfully connected to <https://app.dimensions.ai> (method: dsl.ini file)


#### Search filters

In [35]:
years = [2015, 2016, 2017, 2018, 2019]
years_search = json.dumps(str(years))
FOR_categories = ['11 Medical and Health Sciences']
FOR_categories = json.dumps(FOR_categories)
charite_grid_id = ['grid.6363.0']
pub_type = 

In [36]:
def doi_record(grid_ids, years, FOR_categories):
    dsl = dimcli.Dsl()
    data = dsl.query_iterative(f"""search publications 
    where year in {years} and 
    research_orgs in {json.dumps(grid_ids)} and 
    type="preprint" and 
    journal_lists="PubMed" and
    category_for.name in {FOR_categories}
    return publications[id + doi + pmid + year]""")
    return data.as_dataframe()

#### Get DOIs and dimensions ids for the publications

In [37]:
doi_result = doi_record(charite_grid_id, years, FOR_categories)

1000 / 12680
2000 / 12680
3000 / 12680
4000 / 12680
5000 / 12680
6000 / 12680
7000 / 12680
8000 / 12680
9000 / 12680
10000 / 12680
11000 / 12680
12000 / 12680
12680 / 12680


In [96]:
doi_result.head()

,doi,id,pmid,year
0,10.1002/hep.30925,pub.1120832821,31483871,2019
1,10.1080/20008198.2019.1568134,pub.1112116251,30788063,2019
2,10.1111/jch.13787,pub.1123746530,31891443,2019
3,10.1371/journal.pone.0226843,pub.1123756059,31891612,2019
4,10.1080/20008198.2019.1694766,pub.1122945616,31807235,2019


In [95]:
doi_result.to_csv("../../results/Charite_publication_ids_dimensions_2015_19.csv")

#### Check list of publications for citations/references from policy documents

In [98]:
DOI_list = pd.read_csv("../../results/Charite_publication_ids_dimensions_2015_19.csv")

In [99]:
DOI_list.year.value_counts()

2019    2996
2018    2475
2017    2462
2016    2438
2015    2309
Name: year, dtype: int64

In [100]:
def policy_citations(year, doi_results_table):
    
    doi_results_filtered = doi_results_table[doi_results_table.year==year]
    doi_results_filtered = doi_results_filtered.reset_index() #otherwise the batch slicing does not work correctly
    
    policy_results_list = []
    batch_size = 300
    for idx in range(0, len(doi_results_filtered), batch_size):
        ids_batch = doi_results_filtered.loc[idx:idx+batch_size, "id"].tolist()
        dsl = dimcli.Dsl()
        data = dsl.query(f"""search policy_documents 
            where publication_ids in {json.dumps(ids_batch)} 
            return policy_documents[id + publication_ids]
            limit 500""")
        time.sleep(1.5)
        policy_results_list.append(data.as_dataframe())
        
    policy_results_list_city = pd.concat(policy_results_list, ignore_index=True)
    policy_results_list_city["year"] = year 
    return policy_results_list_city
    

In [103]:
policy_citation_results_list = []
for year in years:
    result = policy_citations(year, DOI_list)
    policy_citation_results_list.append(result)

Returned Policy_documents: 8 (total = 8)
Returned Policy_documents: 8 (total = 8)
Returned Policy_documents: 16 (total = 16)
Returned Policy_documents: 7 (total = 7)
Returned Policy_documents: 5 (total = 5)
Returned Policy_documents: 12 (total = 12)
Returned Policy_documents: 10 (total = 10)
Returned Policy_documents: 6 (total = 6)
Returned Policy_documents: 1 (total = 1)
Returned Policy_documents: 6 (total = 6)
Returned Policy_documents: 2 (total = 2)
Returned Policy_documents: 3 (total = 3)
Returned Policy_documents: 3 (total = 3)
Returned Policy_documents: 6 (total = 6)
Returned Policy_documents: 10 (total = 10)
Returned Policy_documents: 9 (total = 9)
Returned Policy_documents: 0
Returned Policy_documents: 6 (total = 6)
Returned Policy_documents: 2 (total = 2)
Returned Policy_documents: 6 (total = 6)
Returned Policy_documents: 0
Returned Policy_documents: 5 (total = 5)
Returned Policy_documents: 7 (total = 7)
Returned Policy_documents: 2 (total = 2)
Returned Policy_documents: 3 (to

In [104]:
policy_citation_results_list = pd.concat(policy_citation_results_list, ignore_index=True)

C:\Users\riedeln\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [128]:
policy_citation_results_list.to_csv("../../results/policy_citations.csv")